<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Refactoring" data-toc-modified-id="Refactoring-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Refactoring</a></span></li></ul></div>

# Refactoring

As the code may be modified later, I want to refactor from the jupyter notebook to a Python script. Thus I create a file ``main.py``:

In [4]:
import pandas as pd


def clean_csv(filename: str) -> pd.DataFrame:
    df = pd.read_csv(filename)
    return df


if __name__ == "__main__":
    filename = "../input/raw/stackexchange_812k.csv"
    cleaned = clean_csv(filename)
    print(cleaned.head())

   post_id  parent_id  comment_id  \
0        1        NaN         NaN   
1        2        NaN         NaN   
2        3        NaN         NaN   
3        4        NaN         NaN   
4        6        NaN         NaN   

                                                text category  
0                      Eliciting priors from experts    title  
1                                 What is normality?    title  
2  What are some valuable Statistical Analysis op...    title  
3  Assessing the significance of differences in d...    title  
4  The Two Cultures: statistics vs. machine learn...    title  
